In [ ]:
            if True:
                (X, Y) = KF.predict()

                if not len(det):
                    # 칼만필터로 예측된 값을 텐서 변환
                    X = (torch.tensor(X, device="cuda")).view(-1)
                    Y = (torch.tensor(Y, device="cuda")).view(-1)

                    # 예측 좌표와 이전 프레임의 값을 가져와 (1,6)의 텐서로 결합
                    xywh = torch.cat([X, Y, Last_whcc[0:2]], dim=0).tolist()

                    xyxy = (xywh2xyxy(torch.tensor(xywh, device="cuda").view(1, 4))).view(-1)
                    det = torch.cat([xyxy, Last_whcc[2:4]], dim=0).unsqueeze(0)




                for *xyxy, conf, clas in reversed(det):

                    xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4))).view(-1).tolist()
                    conf = conf.view(-1)
                    clas = clas.view(-1)


                    # 지난 프레임의 예측 값을 측정값으로 사용하여
                    # 칼만필터의 추정값을 계산
                    est_x_y = KF.update(Last_x_y)


                    # 칼만필터에 input 형태로 변형
                    Last_x_y = [[xywh[0]], [xywh[1]]]

                    # 칼만필터의 추정값을 텐서에 적용
                    xywh[0] = est_x_y[0, 0]
                    xywh[1] = est_x_y[1, 0]

                    # 높이와 너비값 저장

                    wh = (torch.tensor(xywh[2:4], device="cuda"))

                    Last_whcc = torch.cat([wh, conf, clas], dim=0)

                    # Yolo의 출력 형태로 텐서 길이로 변경

                    xyxy = (xywh2xyxy(torch.tensor(xywh, device="cuda").view(1, 4))).view(-1)

                    # 원래 형태로 만들어주기
                    Esti_X = torch.cat([xyxy, conf, clas], dim=0)

                    # 텐서를 차원을 맞추어 리스트에 저장
                    Esti_X = Esti_X.unsqueeze(0)
                    li.append(Esti_X)

            if li is not None:
                det = sum_tensor(li)